<a href="https://colab.research.google.com/github/SuyashSonawane/Mask-Detector/blob/master/Mask_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step1
>**Downloading Data from Kaggel**

This installs `kaggel cli` tool that will help us to download dataset directly from the command line

In [0]:
! pip install -q kaggle

This is Google Colab specific command, from here we can upload the `kaggle.json` file, which holds our API credentials.

In [0]:
from google.colab import files

files.upload()

By running the below code, a zip will get downloaded in the current directory

In [0]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ahmetfurkandemr/mask-datasets-v1

Unziping Data in the database folder

In [0]:
import os
import zipfile

local_zip = "mask-datasets-v1.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./database')
zip_ref.close()

## Step2
>**Preparing the data**

Getting the directories

In [0]:
train_dir = 'database/Mask_Datasets/Train'
validation_dir = 'database/Mask_Datasets/Validation'

train_mask_dir = os.path.join(train_dir, 'Mask')
train_nomask_dir = os.path.join(train_dir, 'No_mask')
validation_mask_dir = os.path.join(validation_dir, 'Mask')
validation_nomask_dir = os.path.join(validation_dir, 'No_mask')

train_mask_fnames =os.listdir(train_mask_dir)
train_nomask_fnames =os.listdir(train_nomask_dir)
validation_mask_fnames = os.listdir(validation_mask_dir)
validation_nomask_fnames = os.listdir(validation_nomask_dir)

print(len(train_mask_fnames))
print(len(train_nomask_fnames))
print(len(validation_mask_fnames))
print(len(validation_nomask_fnames))

## Step3
> **Now we will start with the Neural Network Tasks**


Due to the small dataset, we will do some image augmentation with TensorFlow's `Image Data Generator`.

We will normalize the images by dividing them by `255`.

We have `10` as batch size and as the images are of different sizes we set the target size as `(28,28)`

As we have a binary classification problem, we have class_mode as `binary`

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 10,
                                                    class_mode = 'binary', 
                                                    target_size = (28, 28))     


validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 10,
                                                          class_mode  = 'binary', 
                                                          target_size = (28, 28))

Now we will structure our Neural Network with TensorFlow's Keras Layers API,
we will define 3 Convolutional Layers each with Max Pooling of shape `(2,2)`.

Then we will compile the model with loss function as `Binary Crossentropy` as we have to do a binary classification , we will use the `RMSprop` Optimizer with learning rate of `10^-4` and accuracy as metrics.

Finally we will print out the model summary



In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])
model.summary()

Below is a callback function which can be used to stop the training once we have the accuracy over `97%`

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.97):
      print("\nReached 97.0% accuracy so cancelling training!")
      self.model.stop_training = True

Now we will train the model with 100 epochs and will save all the training history in the history object 

In [0]:
callback=myCallback()
history = model.fit(
      train_generator,
      steps_per_epoch=75,  
      epochs=100,
      validation_data=validation_generator,validation_steps=35
      )

## Step4
>**Visulazing results**

Now our model is trained and we will picture out the training and validation, accuracy and loss using `Matplotlib` library in python

In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Now we can test our model by uploading pics to the Colab Directory

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

 
  # predicting images
for fn in os.listdir("/content"):
  if(fn.endswith(('.png','.jpg'))):
    path = fn
    img = image.load_img(path, target_size=(28, 28))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(fn)
    print(classes)